In [1]:
'''
Vida modified the existing emission_maps code in the emission repository of foggie to dynamically get:
1. filters: tempreture, density, inflow,outflow, disk, cgm
2. resolution for frb maps
3. units for emission: default: photons s$^{-1}$ cm$^{-2}$ sr$^{-1}$ and ALT: erg s$^{-1}$ cm$^{-2}$ arcsec$^{-2}$
'''


'\nVida modified the existing emission_maps code in the emission repository of foggie to dynamically get:\n1. filters: tempreture, density, inflow,outflow, disk, cgm\n2. resolution for frb maps\n3. units for emission: default: photons s$^{-1}$ cm$^{-2}$ sr$^{-1}$ and ALT: erg s$^{-1}$ cm$^{-2}$ arcsec$^{-2}$\n'

In [2]:
import numpy as np
import yt
import unyt
from yt import YTArray
from yt.data_objects.level_sets.api import * 
import argparse
import os
from astropy.table import Table
from astropy.io import ascii
import multiprocessing as multi


import datetime
from scipy import interpolate
import shutil
import matplotlib.pyplot as plt
import cmasher as cmr
import matplotlib.colors as mcolors
import h5py
import trident

# These imports are FOGGIE-specific files
from foggie.utils.consistency import *
from foggie.utils.get_run_loc_etc import get_run_loc_etc
from foggie.utils.yt_fields import *
from foggie.utils.foggie_load import *
from foggie.utils.analysis_utils import *

# These imports for datashader plots
import datashader as dshader
from datashader.utils import export_image
import datashader.transfer_functions as tf
import pandas as pd
import matplotlib as mpl
import numpy as np
from yt.units.yt_array import YTQuantity
from scipy.ndimage import gaussian_filter


def generate_foggie_paths(halo, run, snap):
    # Define base paths
    foggie_base_dir = "/Users/vidasaeedzadeh/Projects/foggie_data/"
    code_base_path = "/Users/vidasaeedzadeh/Projects/repositories/foggie/foggie/"
    output_base_dir = "/Users/vidasaeedzadeh/Projects/foggie_outputs/"

    # Zero-pad the halo number to 6 digits
    halo_number = halo.zfill(6)

    # Define directory and file paths dynamically
    foggie_dir = os.path.join(foggie_base_dir, f"halo_{halo_number}", run + '/')
    snap_name = os.path.join(foggie_dir, snap, snap)
    halo_c_v_name = os.path.join(code_base_path, f"halo_infos/{halo_number}/{run}/halo_c_v")
    trackname = os.path.join(code_base_path, f"halo_tracks/{halo_number}/nref11n_selfshield_15/halo_track_200kpc_nref9")

    # Output directory (adjust based on needs)
    output_dir = output_base_dir

    # Return paths
    return foggie_dir,code_base_path, snap_name, halo_c_v_name, trackname, output_dir

# specify halo and snapshot
halo = '8508'
run = 'nref11c_nref9f'#'ludicrous/nref11c_nref9f.enhance'
snap = 'DD2520'

foggie_dir,code_path, snap_name, halo_c_v_name, trackname, output_dir = generate_foggie_paths(halo, run, snap)



# System and plotting settings
system = ''  # System you're using
plot = 'emission_FRB'  # Options: emission_map, emission_map_vbins, or emission_FRB or emission_FRB_binsmearing
ions = ['Halpha','C II','C III','C IV', 'O VI']#['Lyalpha', 'Halpha', 'CIII', 'CIV', 'OVI','SiII','SiIII','SiIV','MgII'] 
trident_ions =  ['H I','C II','C III','C IV', 'O VI']
Dragonfly_limit = False
Aspera_limit = False
save_suffix = ""
file_suffix = ""


# Filtering settings (optional)
segmentation_filter='radial_velocity' # for categorizing inflow vs outflow it can also be 'metallicity'
filter_type = None  # Type of filter, e.g., 'temperature', 'density', cgm_disk
filter_value = None  # Value for the filter, e.g., 1e4 for temperature < 1e4 K


/Users/vidasaeedzadeh/miniforge3/envs/foggie/lib/python3.12/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# Add Trident ion fields
def add_ion_fields(ds):
    trident.add_ion_fields(ds, ions=trident_ions)
    return ds

def scale_by_metallicity(values,assumed_Z,wanted_Z):
    # The Cloudy calculations assumed a single metallicity (typically solar).
    # This function scales the emission by the metallicity of the gas itself to
    # account for this discrepancy.
    wanted_ratio = (10.**(wanted_Z))/(10.**(assumed_Z))
    return values*wanted_ratio

def make_Cloudy_table(table_index):
    # This function takes all of the Cloudy files and compiles them into one table
    # for use in the emission functions
    # table_index is the column in the Cloudy output files that is being read.
    # each table_index value corresponds to a different emission line

    # this is the the range and number of bins for which Cloudy was run
    # i.e. the temperature and hydrogen number densities gridded in the
    # Cloudy run. They must match or the table will be incorrect.
    hden_n_bins, hden_min, hden_max = 15, -5, 2 #17, -6, 2 #23, -9, 2
    T_n_bins, T_min, T_max = 51, 3, 8 #71, 2, 8

    hden=np.linspace(hden_min,hden_max,hden_n_bins)
    T=np.linspace(T_min,T_max, T_n_bins)
    table = np.zeros((hden_n_bins,T_n_bins))
    for i in range(hden_n_bins):
            table[i,:]=[float(l.split()[table_index]) for l in open(cloudy_path%(i+1)) if l[0] != "#"]
    return hden,T,table

def make_Cloudy_table_thin(table_index):
    hden_n_bins, hden_min, hden_max = 17, -5, 2
    T_n_bins, T_min, T_max = 51, 3, 8 #71, 2, 8

    hden=np.linspace(hden_min,hden_max,hden_n_bins)
    T=np.linspace(T_min,T_max, T_n_bins)
    table = np.zeros((hden_n_bins,T_n_bins))
    for i in range(hden_n_bins):
            table[i,:]=[float(l.split()[table_index]) for l in open(cloudy_path_thin%(i+1)) if l[0] != "#"]
    return hden,T,table


def _Emission_LyAlpha(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_LA(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10**dia1) * ((10.0**H_N)**2.0)
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.63e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_HAlpha(field, data, unit_system='default'):
    H_N = np.log10(np.array(data['H_nuclei_density']))
    Temperature = np.log10(np.array(data['Temperature']))
    dia1 = bl_HA(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.**dia1) * ((10.**H_N)**2.0)
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 3.03e-12)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")
    
def _Emission_CII_1335(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_CII_1335(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 2.03e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_CIII_977(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_CIII_977(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 2.03e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")
    
def _Emission_CIII_1910(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_CIII_1910(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 2.03e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

def _Emission_CIV_1548(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_CIV_1(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.28e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_OVI(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_OVI_1(H_N, Temperature)
    dia2 = bl_OVI_2(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    dia2[idx] = -200.
    emission_line = ((10.0**dia1) + (10**dia2)) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.92e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_SiIII_1207(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4. * np.pi * 1.65e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


def _Emission_SiII_1814(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4.*np.pi*1.65e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

def _Emission_SiIV_1394(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4.*np.pi*1.65e-11)
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

def _Emission_MgII_2796(field, data, unit_system='default'):
    H_N = np.log10(np.array(data["H_nuclei_density"]))
    Temperature = np.log10(np.array(data["Temperature"]))
    dia1 = bl_SiIII_1207(H_N, Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10.0**dia1) * ((10.0**H_N)**2.0)
    emission_line = scale_by_metallicity(emission_line, 0.0, np.log10(np.array(data['metallicity'])))
    
    if unit_system == 'default':
        emission_line = emission_line / (4.*np.pi*1.65e-11) # what should be instead of 1.65e-11 for MgII? or anyother new element I use?
        return emission_line * ytEmU
    elif unit_system == 'ALT':
        emission_line = emission_line / (4. * np.pi)
        emission_line = emission_line / 4.25e10
        return emission_line * ytEmUALT
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")


In [4]:
## Disk finder and remover

halo_dict = {   '2392'  :  'Hurricane' ,
                '2878'  :  'Cyclone' ,
                '4123'  :  'Blizzard' ,
                '5016'  :  'Squall' ,
                '5036'  :  'Maelstrom' ,
                '8508'  :  'Tempest',
                '002392'  :  'Hurricane' ,
                '002878'  :  'Cyclone' ,
                '004123'  :  'Blizzard' ,
                '005016'  :  'Squall' ,
                '005036'  :  'Maelstrom' ,
                '008508'  :  'Tempest' }

def halo_id_to_name(halo_id):
    return halo_dict[str(halo_id)]
    
def read_virial_mass_file(halo_id,snapshot,refinement_scheme,codedir,key="radius"):
    '''
    Read in single entries from the virial mass file for a certain key for 1 snapshot
    Keys include: ['redshift', 'snapshot', 'radius', 'total_mass', 'dm_mass', 'stars_mass',
    'young_stars_mass', 'old_stars_mass', 'gas_mass', 'gas_metal_mass', 'gas_H_mass',
    'gas_HI_mass', 'gas_HII_mass', 'gas_CII_mass', 'gas_CIII_mass', 'gas_CIV_mass',
    'gas_OVI_mass', 'gas_OVII_mass', 'gas_MgII_mass', 'gas_SiII_mass', 'gas_SiIII_mass', 
    'gas_SiIV_mass', 'gas_NeVIII_mass']
    '''
    from astropy.table import Table
    masses_dir = codedir+"foggie/halo_infos/"+halo_id+"/"+refinement_scheme+"/rvir_masses.hdf5"
    rvir_masses = Table.read(masses_dir, path='all_data')
    
    return rvir_masses[key][rvir_masses['snapshot']==snapshot][-1]
    
    
def get_cgm_density_cut(ds,cut_type="comoving_density",additional_factor=2.,code_dir=None):
    '''
    Get a density cutoff to separate the galaxy from the CGM
    '''
    if cut_type=="comoving_density":
        z = ds.get_parameter('CosmologyCurrentRedshift')
        cgm_density_cut = 0.1 *additional_factor* cgm_density_max * (1+z)**3
    elif cut_type=="relative_density":
        try: Rvir = read_virial_mass_file(gal_id, "RD0042","nref11c_nref9f",code_dir)
        except:
            print("Warning: Could not read rvir file for this halo...")
            Rvir = 300.
        print("Rvir set to:",Rvir)
        #ds, refine_box = foggie_load(snap_name, trackname, halo_c_v_name=halo_c_v_name, do_filter_particles=True,disk_relative=True,particle_type_for_angmom=particle_type_for_angmom,smooth_AM_name = smooth_AM_name)
        sphere = ds.sphere(center=ds.halo_center_kpc, radius=(Rvir, 'kpc')) 
        z = ds.get_parameter('CosmologyCurrentRedshift')
        mask = ((sphere['gas','radius_corrected']>50./(1+z)) & (sphere['gas','density']<=cgm_density_max*(1+z)**3))
        
        mean_density = np.average( sphere['gas','density'][mask], weights=sphere['cell_volume'][mask])
        stdv_density = np.sqrt( np.average( np.power(sphere['gas','density'][mask] - mean_density , 2) , weights=sphere['cell_volume'][mask]))
        cgm_density_cut = mean_density + additional_factor * stdv_density 
    else:
        # Define the density cut between disk and CGM to vary smoothly between 1 and 0.1 between z = 0.5 and z = 0.25,
        # with it being 1 at higher redshifts and 0.1 at lower redshifts
        # Cassi's original definition
        current_time = ds.current_time.in_units('Myr').v
        if (current_time<=7091.48):
            density_cut_factor = 20. - 19.*current_time/7091.48
        elif (current_time<=8656.88):
            density_cut_factor = 1.
        elif (current_time<=10787.12):
            density_cut_factor = 1. - 0.9*(current_time-8656.88)/2130.24
        else:
            density_cut_factor = 0.1
        
        cgm_density_cut = cgm_density_max * density_cut_factor * additional_factor
        z = ds.get_parameter('CosmologyCurrentRedshift')
        print("Cassi's cut is:",cgm_density_cut,"(comoving would be",0.1*additional_factor*cgm_density_max*(1+z)**3,")")
    return cgm_density_cut



def find_disk(ds,max_disk_radius=100.,use_comoving_density=True,comoving_scaler=0.2):
    cgm_density_cut = get_cgm_density_cut(ds,use_comoving_density,comoving_scaler)


    sphere = ds.sphere(center=ds.halo_center_kpc, radius=(max_disk_radius, 'kpc'))
    sph_ism = sphere.cut_region("obj['density'] > %.3e" % (cgm_density_cut))
    

    #Test to identify just the disk
    master_clump = Clump(sphere, ("gas", "density"))
    master_clump.add_validator("min_cells", 200)
    c_min = sph_ism["gas", "density"].min()
    c_max = sph_ism["gas", "density"].max()
    #Force to only find highest level clumps
    step = c_max / c_min #100. #2.0

    find_clumps(master_clump, c_min, c_max, step)

    leaf_clumps=master_clump.leaves
    #child_clumps=master_clump.children
    current_max=0
    for leaf in leaf_clumps:
        ncells = np.size(leaf["gas","density"])
        if ncells>current_max:
            current_max = ncells
            disk_clump = leaf


    return disk_clump



global clump_cell_ids

def _masked_field(field,data):
    return ~np.isin(data["index","cell_id_2"],clump_cell_ids) #if you want to remove disk
    #return np.isin(data["index","cell_id_2"],clump_cell_ids) #if you only want disk
    

global max_gid
global gx_min; global gy_min; global gz_min
global gx_max; global gy_max; global gz_max

def get_cell_grid_ids(field, data):
    gids = data['index','grid_indices'] + 1 #These are different in yt and enzo...
    u_id = np.copy(gids)
    
    idx_dx = data['index','dx']

    x_id = np.divide(data['index','x'] - idx_dx/2. , idx_dx)
    y_id = np.divide(data['index','y'] - idx_dx/2. , idx_dx)
    z_id = np.divide(data['index','z'] - idx_dx/2. , idx_dx)
    
    
    for gid in np.round(np.unique(gids)).astype(int): 
        if gid<=0: continue
        grid_mask = (gids==gid)
        if np.size(np.where(grid_mask)[0])<=0: continue

        gx = x_id[grid_mask]
        gy = y_id[grid_mask]
        gz = z_id[grid_mask]

        gx = gx - gx_min[gid-1]
        gy = gy - gy_min[gid-1]
        gz = gz - gz_min[gid-1]


        max_x = gx_max[gid-1]-gx_min[gid-1]
        max_y = gy_max[gid-1]-gy_min[gid-1]

        c_id =  gx+gy*(max_x+1) +gz*(max_x+1)*(max_y+1)

        u_id[grid_mask] = np.round(gid + c_id * (max_gid+1)).astype(np.uint64)
    return u_id    
    


def load_disk(ds,clump_file,source_cut=None):
    '''
    Function to load a disk cut region defined by clump_finder.py
    Prior to calling this function, the  cell_grid_ids must be added to the dataset
    
    '''

    global clump_cell_ids
    hf = h5py.File(clump_file,'r')
    clump_cell_ids = np.round(np.array(hf['cell_ids']).astype(np.uint64))
    hf.close()

    global max_gid
    max_gid=-1
    for g,m in ds.all_data().blocks:
        if g.id>max_gid: max_gid=g.id
    
    global gx_min; global gy_min; global gz_min
    global gx_max; global gy_max; global gz_max

    gx_min = np.zeros((max_gid))
    gy_min = np.zeros((max_gid))
    gz_min = np.zeros((max_gid))
    gx_max = np.zeros((max_gid))
    gy_max = np.zeros((max_gid))
    gz_max = np.zeros((max_gid))
    
    for g,m in ds.all_data().blocks:
        g_dx = g['index','dx'].max()

        gx_min[g.id-1] = (g['index','x'].min() - g_dx/2.)  / g_dx
        gy_min[g.id-1] = (g['index','y'].min() - g_dx/2.)  / g_dx
        gz_min[g.id-1] = (g['index','z'].min() - g_dx/2.)  / g_dx

        gx_max[g.id-1] = (g['index','x'].max() - g_dx/2.)  / g_dx
        gy_max[g.id-1] = (g['index','y'].max() - g_dx/2.)  / g_dx
        gz_max[g.id-1] = (g['index','z'].max() - g_dx/2.)  / g_dx 

    ds.add_field(
        ('index', 'cell_id_2'),
          function=get_cell_grid_ids,
          sampling_type='cell',
          force_override=True
    )

    ds.add_field(
        ("index","masked_field"),
        function=_masked_field,
        sampling_type="cell",
        units="",
        force_override=True
    )

    if source_cut is None:
        source_cut = ds.all_data()
    masked_region = source_cut.cut_region(["obj['index','masked_field']"])

    return masked_region



In [5]:
def filter_ds(box):
    '''This function filters the yt data object passed in as 'box' into inflow and outflow regions,
    based on metallicity, and returns the box filtered into these regions.'''

    if (segmentation_filter=='metallicity'):
        box_#inflow = box.include_below(('gas','metallicity'), 0.01, 'Zsun')
        box_outflow = box.include_above(('gas','metallicity'), 1., 'Zsun')
        box_neither = box.include_above(('gas','metallicity'), 0.01, 'Zsun')
        box_neither = box_neither.include_below(('gas','metallicity'), 1., 'Zsun')
    elif (segmentation_filter=='radial_velocity'):
        box_inflow = box.include_below(('gas','radial_velocity_corrected'), -100., 'km/s')
        box_outflow = box.include_above(('gas','radial_velocity_corrected'), 200., 'km/s')
        box_neither = box.include_above(('gas','radial_velocity_corrected'), -100., 'km/s')
        box_neither = box_neither.include_below(('gas','radial_velocity_corrected'), 200., 'km/s')

    return box_inflow, box_outflow, box_neither

def make_FRB(ds, refine_box, snap, ions, unit_system='default', filter_type=None, filter_value=None,resolution=100):
    '''This function takes the dataset 'ds' and the refine box region 'refine_box' and
    makes a fixed resolution buffer of surface brightness from edge-on, face-on,
    and arbitrary orientation projections of all ions in the list 'ions'.'''

    halo_name = halo_dict[str(halo)]

    # Determine resolution and bin size
    pix_res = float(np.min(refine_box['dx'].in_units('kpc')))
    bin_size_kpc = resolution*pix_res
    round_bin_size_kpc = round(bin_size_kpc,2)
    # Ensure fov_kpc is in kpc
    if not hasattr(ds.refine_width, 'in_units'):
        fov_kpc = YTQuantity(ds.refine_width, 'kpc')  # Wrap in YTQuantity with units
    else:
        fov_kpc = ds.refine_width.in_units('kpc')  # Convert to kpc if it has units
    
    # Convert to numeric value (without units) for calculations
    fov_kpc_value = fov_kpc.v

    res= int(fov_kpc_value/bin_size_kpc)

    # Print for debugging
    print(f"Native resolution (pix_res): {pix_res:.2f} kpc")
    print(f"Field of view (FOV): {fov_kpc_value:.3f} kpc")
    print(f"Adjusted bin size (bin_size_kpc): {bin_size_kpc:.2f} kpc")
    print(f"Adjusted number of bins (res): {res}")
    
    print('z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1))

    # Apply inflow/outflow or disk/CGM filtering using the filter_ds function if specified
    if filter_type == 'inflow_outflow':
        # Apply the inflow/outflow filtering
        box_inflow, box_outflow, box_neither = filter_ds(ds.all_data())
        data_sources = {'inflow': box_inflow, 'outflow': box_outflow, 'neither': box_neither}

    elif filter_type == 'disk_cgm':
        # Apply the disk/CGM filtering
        box_cgm = load_disk(ds,clump_file,source_cut=None)
        data_sources = {'cgm': box_cgm}
        
    else:
        # Standard filtering or no filter
        data_sources = {'all': ds.all_data()}
        if filter_type and filter_value:
            if filter_type == 'temperature':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
            elif filter_type == 'density':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'density'] > {filter_value})"])
            else:
                raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

    # Define the unit string based on unit_system
    if unit_system == 'default':
        unit_label = '[photons s$^{-1}$ cm$^{-2}$ sr$^{-1}$]'
    elif unit_system == 'ALT':
        unit_label = '[erg s$^{-1}$ cm$^{-2}$ arcsec$^{-2}$]'
    else:
        raise ValueError("Invalid unit_system specified. Use 'default' or 'ALT'.")

    # Create HDF5 file for saving emission maps
    save_path = prefix + f'FRBs/res_{bin_size_kpc:.2f}/' 
    os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
    f = h5py.File(save_path + halo_name + '_emission_maps' + save_suffix + '.hdf5', 'a')
    grp = f.create_group('z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1))
    grp.attrs.create("image_extent_kpc", ds.refine_width)
    grp.attrs.create("redshift", ds.get_parameter('CosmologyCurrentRedshift'))
    grp.attrs.create("halo_name", halo_name)
    grp.attrs.create("emission_units", unit_label)
    grp.attrs.create("gas_density_units", 'g/cm^2')
    grp.attrs.create("stars_density_units", 'Msun/kpc^2')
    grp.attrs.create("bin_size_kpc", round_bin_size_kpc)
    grp.attrs.create("number_of_bins", res)





    # Loop through ions and create projections for each region
    for region, data_source in data_sources.items():
        for ion in ions:
            print(ion)

            #Edge-on projection
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, ('gas', 'Emission_' + ions_dict[ion]),
                                          center=ds.halo_center_kpc, data_source=data_source,width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.z_unit_disk, buff_size=[res, res], method = 'integrate', weight_field=None)
            frb_edge = proj_edge.frb[('gas', 'Emission_' + ions_dict[ion])]
            dset1 = grp.create_dataset(f"{ion}_emission_edge_{region}", data=frb_edge)

            # Set colormap and save projection plot
            mymap = cmr.get_sub_cmap('cmr.flamingo', 0.2, 0.8)
            mymap.set_bad("#421D0F")
            proj_edge.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_edge.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_edge.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission ' + unit_label)
            proj_edge.set_font_size(20)
            proj_edge.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_edge.save(save_path + f'{snap}_{ion}_emission_map_edge-on_{region}' + save_suffix + '.png')

            # Face-on projection
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, ('gas', 'Emission_' + ions_dict[ion]),
                                          center=ds.halo_center_kpc, data_source=data_source,width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.x_unit_disk, buff_size=[res, res],weight_field=None)
            frb_face = proj_face.frb[('gas', 'Emission_' + ions_dict[ion])]
            dset2 = grp.create_dataset(f"{ion}_emission_face_{region}", data=frb_face)

            # Set colormap and save projection plot
            proj_face.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_face.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_face.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission ' + unit_label)
            proj_face.set_font_size(20)
            proj_face.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_face.save(save_path + f'{snap}_{ion}_emission_map_face-on_{region}' + save_suffix + '.png')

    # Close the HDF5 file after saving the datasets
    print('finished')
    f.close()

def emission_map_vbins(ds, snap, ions,unit_system='default', filter_type=None, filter_value=None):
    '''Makes many emission maps for each ion in 'ions', oriented both edge-on and face-on, for each line-of-sight velocity bin.'''

    vbins = np.arange(-500., 550., 50.)  # Velocity bins
    ad = ds.all_data()

    for i in range(len(ions)):
        ion = ions[i]

        # Choose colormap based on ion and emission limits
        if (ion == 'Halpha') and Dragonfly_limit:
            cmap1 = cmr.take_cmap_colors('cmr.flamingo', 9, cmap_range=(0.4, 0.8), return_fmt='rgba')
            cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 3, cmap_range=(0.2, 0.6), return_fmt='rgba')
            cmap = np.hstack([cmap2, cmap1])
            mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
        elif (ion == 'OVI') and Aspera_limit:
            cmap1 = cmr.take_cmap_colors('cmr.flamingo', 4, cmap_range=(0.4, 0.8), return_fmt='rgba')
            cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 6, cmap_range=(0.2, 0.6), return_fmt='rgba')
            cmap = np.hstack([cmap2, cmap1])
            mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
        else:
            mymap = cmr.get_sub_cmap('cmr.flamingo', 0.2, 0.8)
        mymap.set_bad(mymap.colors[0])

        # Loop through each velocity bin
        for v in range(len(vbins) - 1):
            # Filter the data by the current velocity bin
            vbox = ds.cut_region(ad, [f"obj[('gas', 'vx_disk')] > {vbins[v]:.1f}"])
            vbox = ds.cut_region(vbox, [f"obj[('gas', 'vx_disk')] < {vbins[v+1]:.1f}"])

            # Apply filtering if specified (e.g., temperature or density cut)
            if filter_type and filter_value:
                if filter_type == 'temperature':
                    vbox = vbox.cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
                elif filter_type == 'density':
                    vbox = vbox.cut_region([f"(obj['gas', 'density'] > {filter_value})"])
                else:
                    raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

            # Edge-on projection
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), 
                                          center=ds.halo_center_kpc, width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.z_unit_disk, data_source=vbox)
            proj_edge.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_edge.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_edge.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + 'Emission' + unit_label)
            proj_edge.set_font_size(20)
            proj_edge.annotate_title(f'$%d < v_{{\\rm los}} < %d$' % (vbins[v], vbins[v+1]))
            proj_edge.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_edge.save(prefix + 'EmissionMap/' + snap + '_' + ion + '_emission_map_edge-on_vbin' + str(v) + save_suffix + '.png')

            # Face-on projection
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), 
                                          center=ds.halo_center_kpc, width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.x_unit_disk, data_source=vbox)
            proj_face.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_face.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_face.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + 'Emission' + unit_label)
            proj_face.set_font_size(20)
            proj_face.annotate_title(f'$%d < v_{{\\rm los}} < %d$' % (vbins[v], vbins[v+1]))
            proj_face.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_face.save(prefix + 'EmissionMap/' + snap + '_' + ion + '_emission_map_face-on_vbin' + str(v) + save_suffix + '.png')



In [6]:

def make_mass_FRB(ds, refine_box, snap, ions, filter_type=None, filter_value=None, resolution=100):
    '''This function calculates and saves mass FRBs and total mass for each ion.'''

    halo_name = halo_dict[str(halo)]

    # Determine resolution and bin size
    pix_res = float(np.min(refine_box['dx'].in_units('kpc')))
    bin_size_kpc = resolution*pix_res
    round_bin_size_kpc = round(bin_size_kpc,2)

    # Ensure fov_kpc is in kpc
    if not hasattr(ds.refine_width, 'in_units'):
        fov_kpc = YTQuantity(ds.refine_width, 'kpc')  # Wrap in YTQuantity with units
    else:
        fov_kpc = ds.refine_width.in_units('kpc')  # Convert to kpc if it has units
    
    # Convert to numeric value (without units) for calculations
    fov_kpc_value = fov_kpc.v

    res= int(fov_kpc_value/bin_size_kpc)

    # Print for debugging
    print(f"Native resolution (pix_res): {pix_res:.2f} kpc")
    print(f"Field of view (FOV): {fov_kpc_value:.3f} kpc")
    print(f"Adjusted bin size (bin_size_kpc): {bin_size_kpc:.2f} kpc")
    print(f"Adjusted number of bins (res): {res}")

    # Apply filtering (if specified)
    if filter_type == 'inflow_outflow':
        box_inflow, box_outflow, box_neither = filter_ds(ds.all_data())
        data_sources = {'inflow': box_inflow, 'outflow': box_outflow, 'neither': box_neither}
    elif filter_type == 'disk_cgm':
        box_disk, box_cgm = filter_ds_disk_cgm(ds)
        data_sources = {'disk': box_disk, 'cgm': box_cgm}
    else:
        data_sources = {'all': ds.all_data()}
        if filter_type and filter_value:
            if filter_type == 'temperature':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
            elif filter_type == 'density':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'density'] > {filter_value})"])
            else:
                raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

    # Create HDF5 file for saving mass maps
    save_path = prefix + f'FRBs/res_{bin_size_kpc:.2f}/' 
    os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
    f = h5py.File(save_path + halo_name + '_emission_maps' + save_suffix + '.hdf5', 'a')
    # Check if the group already exists
    redshift_group_name = 'z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1)
    if redshift_group_name not in f:
        grp = f.create_group(redshift_group_name)
        grp.attrs.create("image_extent_kpc", ds.refine_width)
        grp.attrs.create("redshift", ds.get_parameter('CosmologyCurrentRedshift'))
        grp.attrs.create("halo_name", halo_name)
        grp.attrs.create("bin_size_kpc", round_bin_size_kpc)
        grp.attrs.create("number_of_bins", res)
    else:
        grp = f[redshift_group_name]  # Open the existing group

    # Compute pixel area in cm^2
    pixel_area_kpc2 = (fov_kpc / res) ** 2  # Pixel area in kpc^2
    pixel_area_cm2 = pixel_area_kpc2.in_units('cm**2')  # Convert to cm^2

    # Loop through ions and create projections for each region
    for region, data_source in data_sources.items():
        for ion in ions:
            print(f"Processing ion: {ion}")

            # Replace mass field with ion density
            density_field = ('gas', ions_density_dict[ion]) 

            # Edge-on projection (surface density)
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, density_field,
                                          center=ds.halo_center_kpc, data_source=data_source,
                                          width=(ds.refine_width, 'kpc'), north_vector=ds.z_unit_disk,
                                          buff_size=[res, res], weight_field=None)
            frb_edge = proj_edge.frb[density_field]  # Surface density in g/cm^2
            frb_edge_mass = (frb_edge * pixel_area_cm2).in_units('Msun') 
            # Compute total mass for edge-on projection
            total_mass_edge = (frb_edge * pixel_area_cm2).sum().in_units('Msun')  # Convert to solar masses

            # Save mass frb and total mass in HDF5
            dset1 = grp.create_dataset(f"{ion}_mass_edge_{region}", data=frb_edge_mass)
            dset1.attrs.create("total_mass_Msun", total_mass_edge)

            # Face-on projection (surface density)
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, density_field,
                                          center=ds.halo_center_kpc, data_source=data_source,
                                          width=(ds.refine_width, 'kpc'), north_vector=ds.x_unit_disk,
                                          buff_size=[res, res], weight_field=None)
            frb_face = proj_face.frb[density_field]  # Surface density in g/cm^2
            frb_face_mass = (frb_face* pixel_area_cm2).in_units('Msun') 
            # Compute total mass for face-on projection
            total_mass_face = (frb_face * pixel_area_cm2).sum().in_units('Msun')  # Convert to solar masses

            # Save surface density and total mass in HDF5
            dset2 = grp.create_dataset(f"{ion}_mass_face_{region}", data=frb_face_mass)
            dset2.attrs.create("total_mass_Msun", total_mass_face)

            print(f"Edge total mass for {ion}: {total_mass_edge}")
            print(f"Face total mass for {ion}: {total_mass_face}")

    # Close the HDF5 file after saving the datasets
    print('Mass FRBs finished')
    f.close()



In [7]:

def load_and_calculate(snap, ions, unit_system='default', filter_type=None, filter_value=None, resolution=100):

    '''Loads the simulation snapshot and makes the requested plots, with optional filtering.'''

    # Load simulation output
    if system == 'pleiades_cassi':
        print('Copying directory to /tmp')
        snap_dir = '/nobackup/clochhaa/tmp/' + halo + '/' + run + '/' + target_dir + '/' + snap
        os.makedirs(snap_dir)
        snap_name = foggie_dir + run_dir + snap + '/' + snap
    else:
        snap_name = foggie_dir + snap + '/' + snap
    print('snap_name',snap_name)
    ds, refine_box = foggie_load(snap_name, trackname, do_filter_particles=True, halo_c_v_name=halo_c_v_name, disk_relative=True, correct_bulk_velocity=True)#, smooth_AM_name=smooth_AM_name)
    zsnap = ds.get_parameter('CosmologyCurrentRedshift')
    add_ion_fields(ds)

    

    # Generate emission maps based on the plot type
    if 'emission_map' in plot:
        if 'vbins' not in plot:
            emission_map(ds, snap, ions, filter_type=filter_type, filter_value=filter_value)
        else:
            # Call velocity-binned emission map function with optional filtering
            emission_map_vbins(ds, snap, ions, unit_system=unit_system, filter_type=filter_type, filter_value=filter_value)
    
    if 'emission_FRB' in plot:
        make_FRB(ds, refine_box, snap, ions, unit_system=unit_system, filter_type=filter_type, filter_value=filter_value, resolution=resolution)
        make_mass_FRB(ds, refine_box, snap, ions, filter_type=filter_type, filter_value=filter_value, resolution=resolution)
        

        


    

if __name__ == "__main__":


    # if ('feedback' in run) and ('track' in run):
    #     foggie_dir = '/nobackup/jtumlins/halo_008508/feedback-track/'
    #     run_dir = run + '/'

    #set the clump file directory
    clump_file = output_dir + 'ions_halo_00' + halo + '/' + run + '/' '/Disk/test_Disk.h5'
    
    # Set directory for output location, making it if necessary
    prefix = output_dir + 'ions_halo_00' + halo + '/' + run + '/'
    if not (os.path.exists(prefix)): os.system('mkdir -p ' + prefix)
    table_loc = prefix + 'Tables/'

    print('foggie_dir: ', foggie_dir)
    catalog_dir = code_path + 'halo_infos/00' + halo + '/' + run + '/'
    halo_c_v_name = catalog_dir + 'halo_c_v'
    #smooth_AM_name = catalog_dir + 'AM_direction_smoothed'

    cloudy_path = "/Users/vidasaeedzadeh/Documents/02-Projects/02-FOGGIE/Cloudy-runs/outputs/test-z0/TEST_z0_HM12_sh_run%i.dat"
    #code_path + "emission/cloudy_z0_selfshield/sh_z0_HM12_run%i.dat"
    # These are the typical units that Lauren uses
    # NOTE: This is a volumetric unit since it's for the emissivity of each cell
    # Emission / surface brightness comes from the projections
    emission_units = 's**-1 * cm**-3 * steradian**-1'
    ytEmU = unyt.second**-1 * unyt.cm**-3 * unyt.steradian**-1

    # These are a second set of units that a lot of observers prefer
    # NOTE: This is a volumetric unit since it's for the emissivity of each cell
    # Emission / surface brightness comes from the projections
    emission_units_ALT = 'erg * s**-1 * cm**-3 * arcsec**-2'
    ytEmUALT = unyt.erg * unyt.second**-1 * unyt.cm**-3 * unyt.arcsec**-2

    ####################################
    ## BEGIN CREATING EMISSION FIELDS ##
    ####################################

    # To make the emissivity fields, you need to follow a number of steps
    # 1. Read in the Cloudy values for a given emission line
    # 2. Create the n_H and T grids that represent the desired range of values
    # 3. Set up interpolation function for the emissivity values across the grids
    #    so the code can use the n_H and T values of a simulation grid cell to
    #    interpolate the correct emissivity value
    # 4. Define the emission field for the line
    # 5. Add the line as a value in yt

    ############################
    # Function to register emission fields with unit options
    def register_emission_field_with_unit(field_name, function, emission_units, unit_system):
        yt.add_field(
            ('gas', field_name),
            units=emission_units if unit_system == 'default' else emission_units_ALT,
            function=lambda field, data: function(field, data, unit_system=unit_system),
            take_log=True,
            force_override=True,
            sampling_type='cell',
        )
    
    ############################
    # Unit system setting (can be passed dynamically)
    unit_system = 'default'  # Change this to 'ALT' as needed
    
    ############################
    # H-Alpha
    hden_pts, T_pts, table_HA = make_Cloudy_table(2)
    hden_pts, T_pts = np.meshgrid(hden_pts, T_pts)
    pts = np.array((hden_pts.ravel(), T_pts.ravel())).T
    
    sr_HA = table_HA.T.ravel()
    bl_HA = interpolate.LinearNDInterpolator(pts, sr_HA)
    register_emission_field_with_unit('Emission_HAlpha', _Emission_HAlpha, emission_units, unit_system)
    
    ############################
    # Ly-Alpha
    hden_pts, T_pts, table_LA = make_Cloudy_table(1)
    sr_LA = table_LA.T.ravel()
    bl_LA = interpolate.LinearNDInterpolator(pts, sr_LA)
    register_emission_field_with_unit('Emission_LyAlpha', _Emission_LyAlpha, emission_units, unit_system)
    ############################
    # CII 1335
    hden_pts, T_pts, table_CII_1335 = make_Cloudy_table(10)
    sr_CII_1335 = table_CII_1335.T.ravel()
    bl_CII_1335 = interpolate.LinearNDInterpolator(pts, sr_CII_1335)
    register_emission_field_with_unit('Emission_CII_1335', _Emission_CII_1335, emission_units, unit_system)
    
    ############################
    # CIII 977
    hden_pts, T_pts, table_CIII_977 = make_Cloudy_table(7)
    sr_CIII_977 = table_CIII_977.T.ravel()
    bl_CIII_977 = interpolate.LinearNDInterpolator(pts, sr_CIII_977)
    register_emission_field_with_unit('Emission_CIII_977', _Emission_CIII_977, emission_units, unit_system)

    ############################
    # CIII 1910
    hden_pts, T_pts, table_CIII_1910 = make_Cloudy_table(9)
    sr_CIII_1910 = table_CIII_1910.T.ravel()
    bl_CIII_1910 = interpolate.LinearNDInterpolator(pts, sr_CIII_1910)
    register_emission_field_with_unit('Emission_CIII_1910', _Emission_CIII_1910, emission_units, unit_system)

    ############################
    # CIV 1548
    hden_pts, T_pts, table_CIV_1 = make_Cloudy_table(3)
    sr_CIV_1 = table_CIV_1.T.ravel()
    bl_CIV_1 = interpolate.LinearNDInterpolator(pts, sr_CIV_1)
    register_emission_field_with_unit('Emission_CIV_1548', _Emission_CIV_1548, emission_units, unit_system)
    
    ############################
    # O VI (1032 and 1037 combined)
    hden_pts, T_pts, table_OVI_1 = make_Cloudy_table(5)
    hden_pts, T_pts, table_OVI_2 = make_Cloudy_table(6)
    sr_OVI_1 = table_OVI_1.T.ravel()
    sr_OVI_2 = table_OVI_2.T.ravel()
    bl_OVI_1 = interpolate.LinearNDInterpolator(pts, sr_OVI_1)
    bl_OVI_2 = interpolate.LinearNDInterpolator(pts, sr_OVI_2)
    register_emission_field_with_unit('Emission_OVI', _Emission_OVI, emission_units, unit_system)
    
    ############################
    # SiIII 1207
    cloudy_path_thin = code_path + "emission/cloudy_z0_HM05/bertone_run%i.dat"
    hden_pts, T_pts, table_SiIII_1207 = make_Cloudy_table_thin(11)
    hden_pts, T_pts = np.meshgrid(hden_pts, T_pts)
    pts = np.array((hden_pts.ravel(), T_pts.ravel())).T
    sr_SiIII_1207 = table_SiIII_1207.T.ravel()
    bl_SiIII_1207 = interpolate.LinearNDInterpolator(pts, sr_SiIII_1207)
    register_emission_field_with_unit('Emission_SiIII_1207', _Emission_SiIII_1207, emission_units, unit_system)


    ############################
    ions_dict = {'Lyalpha':'LyAlpha', 'Halpha':'HAlpha', 'C III':'CIII_1910', 'C II': 'CII_1335',
                 'C IV':'CIV_1548','O VI':'OVI', 'Si III':'SiIII_1207'}
    ions_density_dict = {'Lyalpha':'LyAlpha', 'Halpha':'H_p0_density', 'C III':'C_p2_density','C II':'C_p1_density',
                 'C IV':'C_p3_density','O VI':'O_p5_density', 'Si III':'Si_p2_density', 'Si II':'Si_p1_density', 'Si IV':'Si_p3_density','Mg II':'Mg_p1_density'}
    
    label_dict = {'Lyalpha':r'Ly-$\alpha$', 'Halpha':r'H$\alpha$', 'C III':'C III','C II':'C II',
                'C IV':'C IV','O VI':'O VI', 'Si III':'Si III'}

    if unit_system  == 'default':
        zlim_dict = {'Lyalpha':[1e-1,1e7], 'Halpha':[1e-1,1e6], 'C III':[1e-4,1e2],'C II':[1e-7,1e2],
                 'C IV':[1e-2,1e4], 'O VI':[1e-2,1e4], 'Si III':[1e-1,1e4]}
    elif unit_system == 'ALT':
        zlim_dict = {'Lyalpha':[1e-22,1e-16], 'Halpha':[1e-22,1e-16], 'C III':[1e-26,1e-16],'C II':[1e-26,1e-16],
                 'C IV':[1e-23,1e-16], 'O VI':[1e-23,1e-16], 'Si III':[1e-22,1e16]}
        



resolution_list = [2]#10,50,88]
for reso in resolution_list:
    load_and_calculate(snap, ions, unit_system='default', resolution=reso, filter_type = 'disk_cgm') #'default'or choose 'ALT' for erg unit
    

yt : [INFO     ] 2025-01-15 13:39:56,118 Parameters: current_time              = 639.44151531954
yt : [INFO     ] 2025-01-15 13:39:56,119 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2025-01-15 13:39:56,119 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-01-15 13:39:56,119 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-01-15 13:39:56,119 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2025-01-15 13:39:56,120 Parameters: current_redshift          = 5.0084873179923e-06
yt : [INFO     ] 2025-01-15 13:39:56,120 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2025-01-15 13:39:56,120 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2025-01-15 13:39:56,120 Parameters: omega_radiation           = 0
yt : [INFO     ] 2025-01-15 13:39:56,120 Parameters: hubble_constant           = 0.695


foggie_dir:  /Users/vidasaeedzadeh/Projects/foggie_data/halo_008508/nref11c_nref9f/
snap_name /Users/vidasaeedzadeh/Projects/foggie_data/halo_008508/nref11c_nref9f/DD2520/DD2520
Opening snapshot /Users/vidasaeedzadeh/Projects/foggie_data/halo_008508/nref11c_nref9f/DD2520/DD2520
get_refine_box: using this location:         col1          col2     col3     col4    col5     col6     col7  col8
------------------- -------- -------- ------- -------- -------- ------- ----
4.4408920985006e-16 0.488865 0.470316 0.50854 0.490865 0.472316 0.51054    9


Parsing Hierarchy : 100%|██████████| 7623/7623 [00:00<00:00, 28054.33it/s]
yt : [INFO     ] 2025-01-15 13:39:56,573 Gathering a field list (this may take a moment.)


This snapshot is not in the halo_c_v file, calculating halo center...
get_halo_center: code_length code_velocity
get_halo_center: obtained the spherical region
get_halo_center: extracted the DM density
get_halo_center: we have obtained the preliminary center
got the velocities
get_halo_center: located the main halo at: [0.4898500442504883, 0.47119617462158203, 0.5095453262329102] [unyt_quantity(0.00134392, 'code_velocity'), unyt_quantity(-0.00192745, 'code_velocity'), unyt_quantity(0.00080744, 'code_velocity')]
filtering young_stars particles...
filtering young_stars3 particles...
filtering young_stars8 particles...
filtering old_stars particles...
filtering stars particles...
filtering dm particles...
using particle type  young_stars  to derive angular momentum


yt : [WARNING  ] 2025-01-15 13:40:08,452 Field ('gas', 'H_p0_number_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-01-15 13:40:08,455 Field ('gas', 'H_p0_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-01-15 13:40:08,457 Field ('gas', 'H_p0_mass') already exists. Not clobbering.


found angular momentum vector
Native resolution (pix_res): 0.27 kpc
Field of view (FOV): 287.768 kpc
Adjusted bin size (bin_size_kpc): 0.55 kpc
Adjusted number of bins (res): 524
z=0.0


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/vidasaeedzadeh/Projects/foggie_outputs/ions_halo_008508/nref11c_nref9f//Disk/test_Disk.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)